In [47]:
# __future__ imports
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [48]:
#import two best libraries :D
import numpy as np
import tensorflow as tf

In [49]:
# Download training data from http://download.tensorflow.org/data/iris_training.csv
# Download testing data from http://download.tensorflow.org/data/iris_test.csv
iris_training = "iris_training.csv"
iris_testing = "iris_test.csv"

In [50]:
# import data from csv. Data will be loaded in tensorflow Dataset format.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
                filename = iris_training,
                target_dtype = np.int,
                features_dtype = np.float32)
testing_set = tf.contrib.learn.datasets.base.load_csv_with_header(
                filename = iris_testing,
                target_dtype = np.int,
                features_dtype = np.float32)

In [51]:
# Since we have three classes, we will merge class = 1 and class = 2. 
# So, now we will have binary classification problem (0, 1)

X_train_data = training_set.data
Y_train = training_set.target
Y_train[Y_train > 1] = 1

X_test_data = testing_set.data
Y_test = testing_set.target
Y_test[Y_test > 1] = 1

In [52]:
# tf.placeholder is used for data
X = tf.placeholder(tf.float32, shape = [None, 4])
Y = tf.placeholder(tf.float32, shape = [None])

In [53]:
# tf.Variable is used for Variable
W = tf.Variable(tf.random_normal([4,1]), name="weights" )
b = tf.Variable(tf.random_normal([1]), name = "bias")

In [54]:
# Now let's assume hypothesis. We are assuming function
#      Y = W * X + b
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

In [55]:
# Use categorical cross entropy loss (or cost)
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y)*tf.log(1 - hypothesis))

In [56]:
# We will define an optimizer (Gradient Descent Optimizer)
# learning_rate = 0.001 and we will use it to minimize cost
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.001)
train = optimizer.minimize(cost)

In [57]:
# Predict if output probability > 0.5
predicted = tf.cast(hypothesis > 0.5, dtype= tf.float32)

#Define accuracy
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [58]:
#create a new session
sess = tf.Session()

# Initialize all the variables
sess.run(tf.global_variables_initializer())
    
# train for 20000 epochs
for epoch in range(20000):
        
    cost_val, acc_val, _ = sess.run([cost,accuracy, train], feed_dict = {X : X_train_data, Y : Y_train})
        
    if(epoch % 1000 == 0):
        print(epoch, cost_val, acc_val)

0 10.3275 0.35
1000 0.763835 0.59
2000 0.70735 0.6125
3000 0.676815 0.635
4000 0.662602 0.6475
5000 0.656595 0.6475
6000 0.654172 0.6475
7000 0.653213 0.6475
8000 0.652818 0.6475
9000 0.652646 0.65
10000 0.652551 0.65
11000 0.652492 0.65
12000 0.652438 0.65
13000 0.652395 0.65
14000 0.65235 0.65
15000 0.652316 0.65
16000 0.652272 0.65
17000 0.65223 0.65
18000 0.652189 0.65
19000 0.65215 0.65


In [59]:
_, acc_test = sess.run([predicted, accuracy], feed_dict = {X: X_test_data, Y: Y_test})

In [60]:
print("test accuracy is ", acc_test)

test accuracy is  0.733333


Let's check if we are getting the same accuracy in sklearn logistic regression also.

In [61]:
from sklearn.linear_model import LogisticRegression

In [21]:
# Define a logistic regression
# We will use very small C value because we are not adding any penalty
logistic_regression = LogisticRegression(max_iter=20000, C = 1e-9, verbose = True)

In [22]:
# fit on training data
logistic_regression.fit(X_train_data, Y_train)

[LibLinear]

LogisticRegression(C=1e-09, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=20000, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=True, warm_start=False)

In [62]:
print("test accuracy using sklearn is ", logistic_regression.score(X_test_data, Y_test))

test accuracy using sklearn is  0.733333333333


So, yeah! We did it right in our tensor flow logistic regression

Now let's try and add regularization term in our logistic regression tensor flow model

In [63]:
cost2 = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y)*tf.log(1 - hypothesis)) + tf.reduce_sum(tf.square(W))


In [64]:
optimizer2 = tf.train.GradientDescentOptimizer(learning_rate= 0.001)
train2 = optimizer2.minimize(cost2)

In [65]:
predicted2 = tf.cast(hypothesis > 0.5, dtype= tf.float32)
accuracy2 = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))

In [30]:
sess2 = tf.Session()

sess2.run(tf.global_variables_initializer())
    
for steps in range(20000):
        
    cost_val, acc_val, _ = sess2.run([cost,accuracy, train], feed_dict = {X : X_train_data, Y : Y_train})
        
    if(steps % 1000 == 0):
        print(steps, cost_val, acc_val)

0 3.8765 0.35
1000 0.648193 0.65
2000 0.648045 0.65
3000 0.647981 0.65
4000 0.647944 0.65
5000 0.647928 0.65
6000 0.647914 0.65
7000 0.647903 0.65
8000 0.647887 0.65
9000 0.647879 0.65
10000 0.647868 0.65
11000 0.647854 0.65
12000 0.64785 0.65
13000 0.647835 0.65
14000 0.647826 0.65
15000 0.64782 0.65
16000 0.647808 0.65
17000 0.6478 0.65
18000 0.647793 0.65
19000 0.647784 0.65


In [31]:
_, acc_test2 = sess.run([predicted, accuracy], feed_dict = {X: X_test_data, Y: Y_test})

In [66]:
print("accuracy with regularization is ", acc_test2)

accuracy with regularization is  0.733333


Nope regularization does not improve our accuracy